# Generatiivinen kielimalli

## Tavoitteet
Mallissa käytetään **imdb** elokuva-arvostelu sivun arvosteluja, joiden perusteella malli oppii generoimaan elokuva-arvosteluja. Generoidut arvostelu teivät tule olemaan järkeviä ja niissä on paljon kielellisiä virheitä, mutta mallin idea on vain demonstroida miten kyseinen malli tehtäisiin. Jos mallia kouluttaisi vielä pari päivää, se saataisiin varmasti generoimaan paljon tarkempia arvosteluja.
## Datan kuvaus
Datasetti pitää sisällään yli 170 000 elokuva-arvostelua, jotka on jaettu positiiivisiksi ja negatiivisiksi arvosteluiksi, kyseisistä luokista ei välitetä tämän mallin koulutuksessa. Data on tekstimuodossa, ja siitä pitää vain karsia muutamia merkkejä.

## Datan esikäsittely
Kielimallit ovat hyvin rankkoja malleja kouluttaa `CPU:lla`, joten otamme `GPU` koulutuksen käyttöön, jos laite sitä tukee.

In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


Luomme oman mapping funktion, jolla käsittelemme `imbd` arvostelu tekstit. Mapping poistaa kaikki `<br />` merkit tekstistä, eli kaikki rivinvaihdot, jotka korvataan välilyönnillä. Poistamme myös kaikki unicode merkin komennolla `[^\\x00-\\x7F]+`. Unicode merkit poistetaan, koska laitetta jolla malli on koulutettu käyttää vanhaa versiota **kerasista**, joka ei tue `utf-8` merkkejä.

In [5]:
def mapping(x):
    x = tf.strings.regex_replace(x, "<br />", " ")
    return tf.strings.regex_replace(x, "[^\x00-\x7F]+", "")

Ladataan datasetti, jonka jälkeen arvostelut viedään `mapping` funktion läpi, joka poistaa rivinvaihdon ja `utf-8` merkit.

In [6]:
import tensorflow as tf 
from tensorflow import keras
tf.TF_ENABLE_ONEDNN_OPTS=0
dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(mapping)

Found 100006 files belonging to 1 classes.


## Mallinnus
Luodaan `PositionalEmbedding` ja `TransformerDecoder` luokat. 
`PositionalEmbedding` luokan ideana on toimia `Embedding`-kerroksena, joka enkoodaa tokenit ja niiden sijainnit sanassa. Sijanti on tärkeää tietää lauseiden luonnissa, koska sanojen sijainnit lauseessa voi vaikuttaa suuresti sanan tarkoitukseen.

In [11]:
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):  
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(                          
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)              
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
  
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions                        
 
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
 
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True                     
  
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")                           
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))    
        mult = tf.concat(                                               
            [tf.expand_dims(batch_size, -1),                            
                tf.constant([1, 1], dtype=tf.int32)], axis=0)              
        return tf.tile(mask, mult)          
    
    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        padding_mask = None       
        if mask is not None:                                       
            padding_mask = tf.cast(                                
                mask[:, tf.newaxis, :], dtype="int32")             
            padding_mask = tf.minimum(padding_mask, causal_mask)   
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)                            
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,                           
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

Vektorisoimme tekstin, joka tuottaa  `15 000:nen` kokoisen sanakirjan eniten käytetyistä sanoista. Asetamme myös esimerkki arvostelujen maksimipituudeksi `100` tokenia. Tämä ei tarkoita `100 merkkiä`, vaan `100 sanaa`. Sanat jotka eivät mahtuneet sanakirjaan mukaan tullaan merkkaamaan `[unk]` tekstinä mallin generaatioissa.

In [ ]:
from tensorflow.keras.layers import TextVectorization
  
sequence_length = 100 
vocab_size = 15000                            
text_vectorization = TextVectorization(
    max_tokens=vocab_size,                
    output_mode="int",                        
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

Seuraavassa solussa määritämme funktion, jonka avulla muutamme datasetin sellaiseksi, että voimme alkaa kouluttamaan sitä valitsemallamme mallilla. Funktiolla valitsemme vektorisoidusta lauseesta kaiken paitsi viimeisen arvon, jonka tallennamme `x`-muuttujaan. Valitsimme sitten kaikki paitsi ensimmäisen arvon, ja tallennamme sen `y`-muuttujaan. Funktio palauttaa sitten x:n ja y:n.
Käymme sitten alkuperäisen datasetin läpi `.map`-komennolla jossa käytämme `prepare_lm_dataset`-funktiota joka välissä.

In [ ]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)    
    x = vectorized_sequences[:, :-1]                         
    y = vectorized_sequences[:, 1:]                          
    return x, y
  
lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

Luomme nyt aikaisemmin määritetyillä `PositionalEmbedding` ja `TransformerDecoder` kerroksilla mallin. Malli koostuu input-kerroksesta, `PositionalEmbedding`-kerroksesta, `TransformerDecoder`-kerroksesta, ja lopulta mallissa on `Dense`-kerros softmax-aktivaatiolla, joka toimii output-kerroksena.

In [ ]:
embed_dim = 256 
latent_dim = 2048 
num_heads = 2 

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)       
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

Luomme `tokens_index` sanakirjan, jonka **avain** on `token index` ja **arvona** on sana, johon tokeni viittaa.

`sample_next` funktio on tärkeässä osassa lauseen luonnissa, sitä kutsutaan loopissa kokoamaan lause yksi sana kerrallaan, kunnes lause on rakennettu. Funktio ottaa sisäänsä todennäköisimpiä sanoja lausetta rakentaessa, josta se valitsee `lämpötila` arvon perusteella satunnaisesti todennäköisemmän tokeni indeksin (mitä isompi **lämpötila**, sitä isompi hajonta sanojen välillä on). Funktio palauttaa kyseisen indeksin, jonka sana arvo haetaan `tokens_index` sanakirjasta ja lisätään lauseen perään.

`TextGenerator` luokkaa käytetään jokaisen epochin jälkeen `callback` koukussa luomaan esimerkkitekstejä. Tämä ei vaikuta mitenkään mallin koulutukseen ja helpottaa vain mallin arviointia koulutuksen aikana.

Tallennamme myös mallin tiedostoon `text_gen.keras` käyttämällä `ModelCheckpoint` luokkaa. Tämä mahdollistaa mallin uudelleen käytön myöhemmin, lataamalla kaikki painoarvot tiedostosta.

In [18]:
import numpy as np
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))    
  
def sample_next(predictions, temperature=1.0):                         
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)
  
class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,                                               
                 generate_length,                                      
                 model_input_length,
                 temperatures=(1.,),                                   
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq
  
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt                                     
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])    
                predictions = self.model(tokenized_sentence)           
                next_token = sample_next(predictions[0, i, :], temperature)         
                sampled_token = tokens_index[next_token]               
                sentence += " " + sampled_token                        
            print(sentence)
  
prompt = "This movie"

from keras.callbacks import ModelCheckpoint

callbacks = [
    ModelCheckpoint(filepath="text_gen.keras", save_best_only=True, monitor="loss"),
    TextGenerator(prompt, generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5)),
]     

Seuraavaksi on mallin koulutus. Annamme mallille `lm_dataset` koulutusdatan, sekä ylhäällä määrittämämme kaksi eri `callback` kutsua. Kyseistä mallia on koulutettu `186` epochia, joka johtuu vain ajasta. Ideaalisti mallia pitäisi vielä kouluttaa muutama sata epochia, jotta sein saisi paremmaksi.

In [24]:
model.fit(lm_dataset, epochs=186, callbacks=callbacks)

Epoch 1/250
391/391 [==============================] - ETA: 0s - loss: 3.6200== Generating with temperature 0.2
This movie film is is an a understatement successor its to a roll modern point horror i films still have recommend anyway being the an first independent film horror by movies [UNK] are so filled common that suspend it your makes belief sense in of their proportion darker are aspects gritty of
== Generating with temperature 0.5
This movie episode did is not fine showcase set idiotic in clichs first that when tiny poor little character guy dies princess in [UNK] successful [UNK] who wish fails is free that of is us also would a roll dangerous of clown sticking with around the pretending face to of never a
== Generating with temperature 0.7
This movie time starts from off the with third straight of kid the and week sucking line liking laughing getting loudly laughs now immediately this appears movie on may a be dark denied karloff november memory starting this off family just ho

ResourceExhaustedError: Graph execution error:

Detected at node 'model/dense_2/BiasAdd' defined at (most recent call last):
    File "C:\Users\kaspe\anaconda3\envs\py310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\kaspe\anaconda3\envs\py310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\kaspe\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\kaspe\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\kaspe\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\kaspe\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\U

## Käyttöönotto
Koska malli on tallennettu `text_gen.keras` tiedostoon, sen voi ottaa käyttöön `keras.load_model` funktiolla. Joudut asettamaan `custom_objects` sanakirjan, jonka avulla malli osaa käyttää luomiamme luokkia.

In [19]:
model = keras.models.load_model("text_gen.keras", custom_objects={"PositionalEmbedding": PositionalEmbedding, "TransformerDecoder": TransformerDecoder})

Tässä on hyvin yksinkertainen tapa luoda lauseita. Asetamme lauseen alun mallille, jonka jälkeen generoimme `1.5` lämpötilalla seuraavat `100` sanaa. Kuten näemme ulos tuleva lause on täysin sekava, mutta kyllä niistä välillä hyvät naurut saa aikaan.

In [26]:
sentence = "This horror movie was"
temperature = 1.5

for i in range(100):
    tokenized_sentence = text_vectorization([sentence])
    predictions = model(tokenized_sentence)
    next_token = sample_next(predictions[0, i, :], temperature)
    sampled_token = tokens_index[next_token]
    sentence += " " + sampled_token
print(sentence)

This horror movie was fabulous cult is absolutely it trash amazing awful rock just silly fashionable ridiculous dull cunning fun darn morbid its genetic wicked flick psychopathic book broke 100 typing visually heart mistake suspect anyways ladies major that [UNK] chad crappy cairo belongs dorothy appears blaming every cleese no jason viewer is make disgraceful there zoom a modernday all ripley must edge blown once miss of repulsive by what your scare hype does good started a a use youre big killing in brings unacceptable  a out right body to below parts serbian levels but by rural worse any depravity chevy movie he
